In [1]:
import os
import folium.plugins as plugins
import folium
import pandas as pd

# Rutas relativas simples
locales_csv = "../data/locales_todos.csv"
locales_restauracion_csv = "../data/locales_restauracion.csv"
procesado_csv = "../data/MAPS_locales_procesado.csv"
categorizado_csv = "../data/MAPS_locales_categorizados.csv"
shapefile_path = "../data/Barrios/BARRIOS.shp"
mapa_html = "../outputs/mapa_locales.html"

# Confirmación
print("📄 locales_todos.csv →", locales_csv)
print("📄 BARRIOS.shp →", shapefile_path)
print("💾 Salida HTML →", mapa_html)

📄 locales_todos.csv → ../data/locales_todos.csv
📄 BARRIOS.shp → ../data/Barrios/BARRIOS.shp
💾 Salida HTML → ../outputs/mapa_locales.html


# 🧭 Extracción de Establecimientos de Restauración con Google Maps API

Este script tiene como objetivo **obtener todos los locales de restauración** (restaurantes, bares, cafeterías, pizzerías, etc.) dentro de los **códigos postales de la ciudad de Madrid** utilizando la API de Google Maps Places.

---

## 🎯 Objetivos

- Extraer datos únicamente de establecimientos de **restauración**.
- Superar la limitación de resultados de la API (60 por consulta) utilizando:
  - Paginación (`next_page_token`)
  - Reintentos con diferentes **palabras clave** como "tapas", "pizzería", etc.
- Aplicar un **filtrado temprano e inteligente** para conservar solo los negocios relevantes.
- Generar un archivo CSV limpio y estructurado con los resultados.

---

## 🛠️ Lógica del Script

### 1. Configuración Inicial
- Se carga la clave de la API desde un archivo `.env`.
- Se define la lista de **códigos postales de Madrid** (28001 a 28055).

### 2. Palabras Clave para Restauración
Se utilizan combinaciones de palabras clave que identifican locales de comida:

```python
["restaurante", "bar", "cafetería", "tapas", "pizzería", "asador", "sushi", "bocatería", "bistró", "comida", "burger", "grill", "gastrobar", "trattoria"]

In [2]:
import os
import time
import requests
import pandas as pd
from dotenv import load_dotenv
from pathlib import Path

# 1. Configuración API
load_dotenv()
API_KEY = os.getenv("GOOGLE_API_KEY")

# 2. Códigos postales de Madrid
codigos_postales = [f'280{str(i).zfill(2)}' for i in range(1, 56)]

# 3. Palabras clave para detectar "restauración"
KEYWORDS = [
    "restaurante", "bar", "cafetería", "tapas", "pizzería",
    "asador", "sushi", "bocatería", "bistró", "comida", "burger",
    "grill", "gastrobar", "trattoria"
]
TYPES_RELEVANTES = {"restaurant", "food", "bar", "cafe"}

# 4. Funciones auxiliares
def get_coordinates(postal_code):
    url = "https://maps.googleapis.com/maps/api/geocode/json"
    params = {'address': f"{postal_code}, Spain", 'key': API_KEY}
    res = requests.get(url, params=params).json()
    if res['status'] != 'OK':
        raise Exception(f"Geocoding error for {postal_code}: {res['status']}")
    location = res['results'][0]['geometry']['location']
    return location['lat'], location['lng']

def is_restauracion(name, types):
    name = name.lower()
    if any(kw in name for kw in KEYWORDS):
        return True
    if any(t in TYPES_RELEVANTES for t in types):
        return True
    return False

def get_places(lat, lng, radius=2000, keyword=None):
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {
        'location': f'{lat},{lng}',
        'radius': radius,
        'key': API_KEY,
        'type': 'restaurant'
    }
    if keyword:
        params['keyword'] = keyword

    all_results = []
    while True:
        response = requests.get(url, params=params).json()
        all_results.extend(response.get("results", []))
        if 'next_page_token' in response:
            time.sleep(2)
            params['pagetoken'] = response['next_page_token']
        else:
            break
    return all_results

def clean_data(places, postal_code):
    data = []
    for p in places:
        name = p.get("name", "")
        types = p.get("types", [])
        if not is_restauracion(name, types):
            continue  # Filtro temprano
        data.append({
            "nombre": name,
            "direccion": p.get("vicinity"),
            "latitud": p.get("geometry", {}).get("location", {}).get("lat"),
            "longitud": p.get("geometry", {}).get("location", {}).get("lng"),
            "codigo_postal": postal_code,
            "types": ", ".join(types),
            "rating": p.get("rating"),
            "user_ratings_total": p.get("user_ratings_total"),
        })
    return pd.DataFrame(data)

# 5. Ejecución por código postal
todos_los_locales = pd.DataFrame()
for cp in codigos_postales:
    try:
        print(f"📍 Procesando CP: {cp}")
        lat, lng = get_coordinates(cp)

        # Búsqueda básica
        lugares = get_places(lat, lng)
        df = clean_data(lugares, cp)

        # Si hay pocos resultados, repetir con keywords adicionales
        if len(df) < 30:
            for kw in ["tapas", "bar", "pizzería"]:
                lugares_kw = get_places(lat, lng, keyword=kw)
                df_extra = clean_data(lugares_kw, cp)
                df = pd.concat([df, df_extra], ignore_index=True)
                if len(df) >= 60:
                    break

        todos_los_locales = pd.concat([todos_los_locales, df], ignore_index=True)
        print(f"✅ {len(df)} locales válidos en {cp}")
        time.sleep(1)

    except Exception as e:
        print(f"⚠️ Error en CP {cp}: {e}")

# 6. Guardar resultados
todos_los_locales.to_csv("../data/locales_restauracion.csv", index=False)
print("✅ Archivo guardado como 'locales_restauracion.csv' en la carpeta 'data'")

📍 Procesando CP: 28001
✅ 60 locales válidos en 28001
📍 Procesando CP: 28002
✅ 60 locales válidos en 28002
📍 Procesando CP: 28003
✅ 60 locales válidos en 28003
📍 Procesando CP: 28004
✅ 60 locales válidos en 28004
📍 Procesando CP: 28005
✅ 60 locales válidos en 28005
📍 Procesando CP: 28006
✅ 60 locales válidos en 28006
📍 Procesando CP: 28007
✅ 60 locales válidos en 28007
📍 Procesando CP: 28008
✅ 60 locales válidos en 28008
📍 Procesando CP: 28009
✅ 60 locales válidos en 28009
📍 Procesando CP: 28010
✅ 60 locales válidos en 28010
📍 Procesando CP: 28011
✅ 60 locales válidos en 28011
📍 Procesando CP: 28012
✅ 60 locales válidos en 28012
📍 Procesando CP: 28013
✅ 60 locales válidos en 28013
📍 Procesando CP: 28014
✅ 60 locales válidos en 28014
📍 Procesando CP: 28015
✅ 60 locales válidos en 28015
📍 Procesando CP: 28016
✅ 60 locales válidos en 28016
📍 Procesando CP: 28017
✅ 60 locales válidos en 28017
📍 Procesando CP: 28018
✅ 60 locales válidos en 28018
📍 Procesando CP: 28019
✅ 60 locales válidos en